In [1]:
import numpy
import random
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import datetime
from datetime import date
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from PIL import Image
import pandas
from sklearn.linear_model import LinearRegression, LogisticRegression
from mpl_toolkits.mplot3d import Axes3D
import seaborn
import scipy
from scipy import stats
from scipy.stats import poisson, ttest_ind
from scipy import linalg
import xarray as xr
import os
import netCDF4
import cftime

In [10]:
Diri = "data/processed/"

In [3]:
# Define Directory
getDiri = "data/"
#
# Get Files
Precip_Files = os.listdir(getDiri+"spatial-g025-pr/") ## These data must be acquired from the LongRunMIP team
print (Precip_Files)
Temp_Files = os.listdir(getDiri+"orig/tas/global/")
print (Temp_Files)

['pr_mon_ECHAM5MPIOM_control_100_g025.nc', 'pr_mon_IPSLCM5A_control_1000_g025.nc', 'pr_ann_HadGEM2ES_control_239_g025.nc', 'pr_mon_CNRMCM61_control_2000_g025.nc', 'pr_mon_MPIESM12_control_1237_g025.nc', 'pr_ann_MPIESM11_control_2000_g025.nc', 'pr_mon_MPIESM12_abrupt4x_999_g025.nc', 'pr_mon_GISSE2R_control_5225_g025.nc', 'pr_mon_ECHAM5MPIOM_abrupt4x_1001_g025.nc', 'pr_mon_HadCM3L_control_1000_g025.nc', 'pr_ann_MPIESM11_abrupt4x_4459_g025.nc', 'pr_mon_CESM104_abrupt4x_5900_g025.nc', 'pr_mon_GISSE2R_abrupt4x_5001_g025.nc', 'pr_mon_CCSM3_abrupt4x_2120_g025.nc', 'pr_ann_HadGEM2ES_abrupt4x_1299_g025.nc', 'pr_mon_CESM104_control_1000_g025.nc', 'pr_ann_FAMOUS_abrupt4x_3000_g025.nc', 'pr_mon_HadCM3L_abrupt4x_1000_g025.nc', 'pr_ann_FAMOUS_control_3000_g025.nc', 'pr_mon_CNRMCM61_abrupt4x_1850_g025.nc', 'pr_mon_CCSM3_control_1530_g025.nc', 'pr_mon_IPSLCM5A_abrupt4x_1000_g025.nc']
['tas_CCSM3_control_1530.nc', 'tas_CCSM3II_control_3805.nc', 'tas_CESM104_control_1000.nc', 'tas_FAMOUS_control_3000.nc

In [4]:
# Open Precipitation Files
Count = 0
for File in Precip_Files:
    Count += 1
    Parts = File.split("_")
    Freq = Parts[1]
    Model = Parts[2]
    Experiment = Parts[3]
    Time = Parts[4]
    print (Freq, Model, Experiment, Time)
print (Count)

mon ECHAM5MPIOM control 100
mon IPSLCM5A control 1000
ann HadGEM2ES control 239
mon CNRMCM61 control 2000
mon MPIESM12 control 1237
ann MPIESM11 control 2000
mon MPIESM12 abrupt4x 999
mon GISSE2R control 5225
mon ECHAM5MPIOM abrupt4x 1001
mon HadCM3L control 1000
ann MPIESM11 abrupt4x 4459
mon CESM104 abrupt4x 5900
mon GISSE2R abrupt4x 5001
mon CCSM3 abrupt4x 2120
ann HadGEM2ES abrupt4x 1299
mon CESM104 control 1000
ann FAMOUS abrupt4x 3000
mon HadCM3L abrupt4x 1000
ann FAMOUS control 3000
mon CNRMCM61 abrupt4x 1850
mon CCSM3 control 1530
mon IPSLCM5A abrupt4x 1000
22


In [5]:
# Create List of Models
Model_List = ['MPIESM12', 'GISSE2R', 'IPSLCM5A', 'CCSM3', 'HadGEM2ES', 'CESM104', \
'ECHAM5MPIOM', 'FAMOUS', 'CNRMCM61', 'HadCM3L', 'MPIESM11']

In [6]:
# Open Temperature Files
for File in Temp_Files:
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    Time = Parts[3].split(".")[0]
    if Model not in Model_List:
        Temp_Files.remove(File)
    else:
        if Experiment == "abrupt4x" or Experiment == "control":
            print (Model, Experiment, Time)
        else:
            Temp_Files.remove(File)
print (Temp_Files)

CCSM3 control 1530
FAMOUS control 3000
MPIESM12 abrupt4x 1000
GISSE2R abrupt4x 5001
ECHAM5MPIOM control 100
IPSLCM5A abrupt4x 1000
MPIESM12 control 1237
CNRMCM61 control 2000
CCSM3 abrupt4x 2120
IPSLCM5A control 1000
CESM104 abrupt4x 5900
MPIESM11 control 2000
HadCM3L control 1000
ECHAM5MPIOM abrupt4x 1001
FAMOUS abrupt4x 3000
CNRMCM61 abrupt4x 1850
HadCM3L abrupt4x 1000
MPIESM11 abrupt4x 4459
['tas_CCSM3_control_1530.nc', 'tas_CESM104_control_1000.nc', 'tas_FAMOUS_control_3000.nc', 'tas_MPIESM12_abrupt4x_1000.nc', 'tas_GISSE2R_abrupt4x_5001.nc', 'tas_ECHAM5MPIOM_control_100.nc', 'tas_IPSLCM5A_abrupt4x_1000.nc', 'tas_MPIESM12_control_1237.nc', 'tas_CNRMCM61_control_2000.nc', 'tas_CCSM3_abrupt4x_2120.nc', 'tas_GISSE2R_control_5525.nc', 'tas_IPSLCM5A_control_1000.nc', 'tas_CESM104_abrupt4x_5900.nc', 'tas_MPIESM11_control_2000.nc', 'tas_CCSM3II_abrupt1400ppm_3132.nc', 'tas_HadCM3L_control_1000.nc', 'tas_ECHAM5MPIOM_abrupt4x_1001.nc', 'tas_FAMOUS_abrupt4x_3000.nc', 'tas_CNRMCM61_abrupt4x_1

In [7]:
# Create Lists of Frequency, 4x Time and Control Time
Freq_List = [" ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " "]
Time_4x_List = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Time_Control_List = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#
# Insert Info Into DataFrame
for File in Precip_Files:
    Parts = File.split("_")
    Freq = Parts[1]
    Model = Parts[2]
    Experiment = Parts[3]
    Time = Parts[4]
    for i in range(len(Model_List)):
        if Model == Model_List[i]:
            Freq_List[i] = Freq
            if Experiment == "abrupt4x":
                Time_4x_List[i] = Time
            elif Experiment == "control":
                Time_Control_List[i] = Time
#
# Create DataFrame to Store Info
Model_DF = pandas.DataFrame({"Model": Model_List, "Frequency": Freq_List, \
"Abrupt4x Time": Time_4x_List, "Control Time": Time_Control_List})
Model_DF

,Model,Frequency,Abrupt4x Time,Control Time
0,MPIESM12,mon,999,1237
1,GISSE2R,mon,5001,5225
2,IPSLCM5A,mon,1000,1000
3,CCSM3,mon,2120,1530
4,HadGEM2ES,ann,1299,239
5,CESM104,mon,5900,1000
6,ECHAM5MPIOM,mon,1001,100
7,FAMOUS,ann,3000,3000
8,CNRMCM61,mon,1850,2000
9,HadCM3L,mon,1000,1000


In [8]:
# Create Precip Control Dictionary
Precip_Cont_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    Precip_Cont_Dict[Model] = i
    i += 1
Precip_Cont_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': 2,
 'CCSM3': 3,
 'HadGEM2ES': 4,
 'CESM104': 5,
 'ECHAM5MPIOM': 6,
 'FAMOUS': 7,
 'CNRMCM61': 8,
 'HadCM3L': 9,
 'MPIESM11': 10}

In [9]:
# Create Precip Anomaly Dictionary
Precip_Anom_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    Precip_Anom_Dict[Model] = i
    i += 1
Precip_Anom_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': 2,
 'CCSM3': 3,
 'HadGEM2ES': 4,
 'CESM104': 5,
 'ECHAM5MPIOM': 6,
 'FAMOUS': 7,
 'CNRMCM61': 8,
 'HadCM3L': 9,
 'MPIESM11': 10}

In [12]:
%%time
# Open Precip Datasets
i = 0
for Model in Model_List:
    print (Model)
# Find Filenames
    Precip_4x_Filename = "pr_"+Freq_List[i]+"_"+Model+"_"+"abrupt4x"+"_"+Time_4x_List[i]+"_g025.nc"
    Precip_Control_Filename = "pr_"+Freq_List[i]+"_"+Model+"_"+"control"+"_"+Time_Control_List[i]+"_g025.nc"
#
# Open 4x and Control Datasets 0-999 Years
    if Model == "MPIESM12" or Model == "IPSLCM5A" or Model == "CCSM3" or Model == "HadGEM2ES":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr
    if Model == "GISSE2R":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.DatetimeNoLeap(1850,1,1), cftime.DatetimeNoLeap(2849,12,31)))
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.DatetimeNoLeap(3981,1,1), cftime.DatetimeNoLeap(4980,12,31)))
    if Model == "CESM104":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.DatetimeNoLeap(420,1,1), cftime.DatetimeNoLeap(1419,12,31)))
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr
    if Model == "FAMOUS":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
    if Model == "CNRMCM61":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr.sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    if Model == "ECHAM5MPIOM":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr
        Precip_4x_New_Time = []
        for k in range(len(Precip_4x_Data)):
            Precip_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        Precip_4x_Data = Precip_4x_Data.assign_coords(time=("time", Precip_4x_New_Time))
        Precip_Control_New_Time = []
        for k in range(len(Precip_Control_Data)):
            Precip_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        Precip_Control_Data = Precip_Control_Data.assign_coords(time=("time", Precip_Control_New_Time))
    if Model == "HadCM3L":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr
        Precip_4x_New_Time = []
        for k in range(len(Precip_Control_Data)):
            Precip_4x_New_Time.append((k + 0.5) / 12)
        Precip_4x_Data = Precip_4x_Data.assign_coords(month=("month", Precip_4x_New_Time))
        Precip_Control_New_Time = (numpy.array(Precip_Control_Data.month) - 0.7) / 12
        Precip_Control_Data = Precip_Control_Data.assign_coords(month=("month", Precip_Control_New_Time))
    if Model == "MPIESM11":
        Precip_4x_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_4x_Filename, use_cftime=True).pr
        Precip_Control_Data = xr.open_dataset(getDiri+"spatial-g025-pr/"+Precip_Control_Filename, use_cftime=True).pr
        Precip_4x_New_Time = []
        for k in range(len(Precip_4x_Data)):
            Precip_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        Precip_4x_Data = Precip_4x_Data.assign_coords(time=("time", Precip_4x_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        Precip_Control_New_Time = []
        for k in range(len(Precip_Control_Data)):
            Precip_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        Precip_Control_Data = Precip_Control_Data.assign_coords(time=("time", Precip_Control_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    print ("Step A")
#
# Calculate Annual Abrupt4x and Control Data
    if Model != "HadCM3L":
        Precip_4x_Annual = Precip_4x_Data.groupby('time.year').mean(dim='time')
        Precip_Control_Annual = Precip_Control_Data.groupby('time.year').mean(dim='time')
    else:
        Month_Bins = numpy.arange(0,int(Time_4x_List[i])+1,1)
        Precip_4x_Bins = Precip_4x_Data.groupby_bins('month', Month_Bins).mean("month")
        Precip_4x_Years = []
        for k in range(len(Precip_4x_Bins)):
            Precip_4x_Years.append(k)
        Precip_4x_Annual = Precip_4x_Bins.assign_coords(month_bins=("month_bins", Precip_4x_Years))
        Precip_4x_Annual = Precip_4x_Annual.rename({"month_bins": "year"})
        Precip_Control_Bins = Precip_Control_Data.groupby_bins('month', Month_Bins).mean("month")
        Precip_Control_Years = []
        for k in range(len(Precip_Control_Bins)):
            Precip_Control_Years.append(k)
        Precip_Control_Annual = Precip_Control_Bins.assign_coords(month_bins=("month_bins", Precip_Control_Years))
        Precip_Control_Annual = Precip_Control_Annual.rename({"month_bins": "year"})
    print ("Step B")
    if Model != "HadCM3L":
        try:
            Precip_Control_Mean = Precip_Control_Data.mean(dim='time') + Precip_4x_Annual * 0 
        except:
            Precip_Control_Mean = Precip_Control_Data.mean(dim='year') + Precip_4x_Annual * 0
    else:
        Precip_Control_Mean = Precip_Control_Data.mean(dim='month') + Precip_4x_Annual * 0
    print ("Step C")
#
# Edit Year to Start at 0 for Abrupt 4x and Control Data
    if Model == "MPIESM12" or Model == "CNRMCM61" or Model == "ECHAM5MPIOM" or Model == "MPIESM11":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 1850
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 1850
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 1850
    if Model == "IPSLCM5A":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 1850
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 1800
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 1850
    if Model == "GISSE2R":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 1850
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 3981
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 1850
    if Model == "CCSM3":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 1
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 1
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 1
    if Model == "HadGEM2ES":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 1860
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 1860
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 1860
    if Model == "CESM104":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 420
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 420
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 420
    if Model == "FAMOUS":
        Precip_4x_Annual['year'] = Precip_4x_Annual['year'] - 5002
        Precip_Control_Annual['year'] = Precip_Control_Annual['year'] - 5002
        Precip_Control_Mean['year'] = Precip_Control_Mean['year'] - 5002
    print ("Step D")
#
# Calculate Precip Anomaly
    Precip_Control = Precip_Control_Annual.sel(year=slice(0,999))
    Precip_Anomaly = Precip_4x_Annual.sel(year=slice(0,999)) - Precip_Control_Mean.sel(year=slice(0,999))
    print ("Step E")
#
# Convert Precip Data to W m^-2
    L_Vapour = 2.5 * 10**6
    Precip_Control = Precip_Control * L_Vapour
    Precip_Anomaly = Precip_Anomaly * L_Vapour
#
# Create New Net CDF Files
    Filename_1 = Diri+"Net_CDF/"+"Precip_Anom_"+Model+".nc"
    Precip_Anomaly.to_netcdf(path=Filename_1)
    Filename_2 = Diri+"Net_CDF/"+"Precip_Control_"+Model+".nc"
    Precip_Control.to_netcdf(path=Filename_2)
    print ("Step F")
    i += 1

MPIESM12
Step A
Step B
Step C
Step D
Step E
Step F
GISSE2R
Step A
Step B
Step C
Step D
Step E
Step F
IPSLCM5A
Step A
Step B
Step C
Step D
Step E
Step F
CCSM3
Step A
Step B
Step C
Step D
Step E
Step F
HadGEM2ES
Step A
Step B
Step C
Step D
Step E
Step F
CESM104
Step A
Step B
Step C
Step D
Step E
Step F
ECHAM5MPIOM
Step A
Step B
Step C
Step D
Step E
Step F
FAMOUS
Step A
Step B
Step C
Step D
Step E
Step F
CNRMCM61
Step A
Step B
Step C
Step D
Step E
Step F
HadCM3L
Step A
Step B
Step C
Step D
Step E
Step F
MPIESM11
Step A
Step B
Step C
Step D
Step E
Step F
CPU times: user 7min 46s, sys: 1min 8s, total: 8min 55s
Wall time: 9min 12s


In [13]:
Filename = "Precip_Anom_MPIESM12.nc"
Precip_4x_Data = xr.open_dataset(Diri+"Net_CDF/"+Filename, use_cftime=True)
Precip_4x_Data

<xarray.Dataset> Size: 41MB
Dimensions:  (lon: 144, lat: 72, year: 1000)
Coordinates:
  * lon      (lon) float64 1kB 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float64 576B -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * year     (year) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    pr       (year, lat, lon) float32 41MB ...

In [14]:
# Create Temp Control Dictionary
Temp_Cont_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    Temp_Cont_Dict[Model] = i
    i += 1
Temp_Cont_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': 2,
 'CCSM3': 3,
 'HadGEM2ES': 4,
 'CESM104': 5,
 'ECHAM5MPIOM': 6,
 'FAMOUS': 7,
 'CNRMCM61': 8,
 'HadCM3L': 9,
 'MPIESM11': 10}

In [15]:
# Create Temp Anomaly Dictionary
Temp_Anom_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    Temp_Anom_Dict[Model] = i
    i += 1
Temp_Anom_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': 2,
 'CCSM3': 3,
 'HadGEM2ES': 4,
 'CESM104': 5,
 'ECHAM5MPIOM': 6,
 'FAMOUS': 7,
 'CNRMCM61': 8,
 'HadCM3L': 9,
 'MPIESM11': 10}

In [16]:
%%time
# Open Temp Datasets
i = 0
for Model in Model_List:
    print (Model)
# Find Filenames
    Temp_4x_Filename = "tas_"+Model+"_"+"abrupt4x"+"_"+Time_4x_List[i]+".nc"
    Temp_Control_Filename = "tas_"+Model+"_"+"control"+"_"+Time_Control_List[i]+".nc"
    if Model == "MPIESM12":
        Temp_4x_Filename = "tas_"+Model+"_"+"abrupt4x"+"_"+"1000"+".nc"
    if Model == "GISSE2R":
        Temp_Control_Filename = "tas_"+Model+"_"+"control"+"_"+"5525"+".nc"
    if Model == "HadGEM2ES":
        Temp_4x_Filename = "tas_"+"HadGEM2"+"_"+"abrupt4x"+"_"+Time_4x_List[i]+".nc"
        Temp_Control_Filename = "tas_"+"HadGEM2"+"_"+"control"+"_"+Time_Control_List[i]+".nc"
#
# Open 4x and Control Datasets
    Temp_4x_Data = xr.open_dataset(getDiri+"orig/tas/global/"+Temp_4x_Filename, use_cftime=True).tas
    Temp_Control_Data = xr.open_dataset(getDiri+"orig/tas/global/"+Temp_Control_Filename, use_cftime=True).tas
    Years_4x = range(len(Temp_4x_Data))
#
# Calculate Annual Abrupt4x and Control Data
    Temp_4x = numpy.array(Temp_4x_Data)
    Temp_Control_Mean = float(Temp_Control_Data.mean()) + Temp_4x*0
#
# Calculate Temp Anomaly
    Temp_Control = Temp_Control_Data[0:1000]
    Temp_Anomaly = Temp_4x[0:1000] - Temp_Control_Mean[0:1000]
#
# Store Info Into DataFrame
    print (numpy.array(Temp_Control))
    if i == 0:
        Temp_Anomaly_DF = pandas.DataFrame({"MPIESM12": Temp_Anomaly})
        Temp_Control_DF = pandas.DataFrame({"MPIESM12": Temp_Control})
    else:
        Temp_Anomaly_DF[Model] = Temp_Anomaly
        Temp_Control_2_DF = pandas.DataFrame({Model: Temp_Control})
        Temp_Control_DF = pandas.concat([Temp_Control_DF, Temp_Control_2_DF], ignore_index=True, axis=1)
    i += 1
#
# Create New CSV Files
Filename_1 = Diri+"CSV/"+"Temp_Anom.csv"
Temp_Anomaly_DF.to_csv(Filename_1)
Filename_2 = Diri+"CSV/"+"Temp_Control.csv"
Temp_Control_DF.to_csv(Filename_2)

MPIESM12
[287.0706  287.00793 287.0309  287.0869  286.94925 286.9248  286.97064
 287.044   287.05872 287.31223 287.08704 286.9412  286.8608  286.97385
 287.06992 287.34055 287.29755 286.98    286.72418 286.7274  286.7254
 286.7904  286.87192 286.8951  286.92337 286.94528 286.8522  286.90887
 286.94366 286.9475  286.91953 286.9766  287.0248  287.11493 286.96988
 286.94766 286.92325 286.94928 287.098   287.17804 287.1145  287.07907
 286.85275 286.786   286.89935 286.8786  286.92398 286.89185 286.9004
 286.95953 286.94955 287.00934 286.9138  286.9879  286.9109  286.82
 286.67084 286.7842  287.07983 286.9163  286.76135 286.85706 286.83743
 286.77686 286.9293  286.80173 286.82413 287.0491  287.08502 286.7858
 286.78528 286.87952 286.89948 287.09225 287.1761  287.08636 286.7831
 286.76944 286.8006  286.85117 287.038   286.93442 286.89362 286.96982
 286.99985 286.99167 287.00388 286.9391  286.8311  286.7432  286.98145
 286.92072 286.84988 286.87253 286.98184 287.1872  287.25113 286.9432
 286.

In [18]:
Temp_Anomaly_DF

,MPIESM12,GISSE2R,IPSLCM5A,CCSM3,HadGEM2ES,CESM104,ECHAM5MPIOM,FAMOUS,CNRMCM61,HadCM3L,MPIESM11
0,1.360443,1.199829,1.275909,2.276764,1.201233,1.180817,0.780731,1.506622,1.241150,1.689056,0.984650
1,1.987671,1.860046,1.774536,2.561035,1.818787,1.598694,1.877838,2.441376,1.924286,2.178467,1.628540
2,2.465454,2.060455,2.036194,2.729034,2.296387,1.979828,2.420410,3.106506,2.523804,2.515228,2.226990
3,2.726227,2.172089,2.390839,2.638702,2.574860,2.197510,2.496979,3.668945,2.965424,2.852692,2.487976
4,2.841034,2.414581,2.742126,2.746277,2.856323,2.426361,2.649048,4.191986,3.296051,3.271484,2.941010
...,...,...,...,...,...,...,...,...,...,...,...
995,6.196198,4.402954,7.308472,4.869171,8.093628,5.850616,9.957733,12.895905,8.748474,5.696136,6.334045
996,6.239532,4.355042,7.180603,4.871216,8.264740,6.178955,10.167511,13.071625,8.803833,5.740356,6.383240
997,6.298920,4.353790,7.247498,4.817291,8.076172,6.251648,10.105499,12.850891,8.920532,5.873718,6.358185
998,6.541870,4.309937,7.279419,4.867218,8.073578,6.007782,10.130524,12.844543,8.882629,5.926819,6.199738
